In [19]:
# Cell 1: Environment, imports, paths, IST helpers
# --- SAFE version for Windows ---
import sys, subprocess
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
subprocess.run([sys.executable, "-m", "pip", "install", "textblob", "pycountry", "plotly"])


import os, re, json
from datetime import datetime, time
import pandas as pd
import numpy as np
import pytz
from textblob import TextBlob
import plotly.express as px
import plotly.graph_objects as go
import pycountry

# Optional: VADER (best-effort)
use_vader = True
try:
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    import nltk
    nltk.download('vader_lexicon', quiet=True)
    sia = SentimentIntensityAnalyzer()
except Exception:
    use_vader = False
    sia = None

# File paths (adjust to your machine if different)
PLAYSTORE_PATH = 'C:\\Users\\hprus\\OneDrive\\Desktop\\Null Class\\Play Store Data.csv'
REVIEWS_PATH = 'C:\\Users\\hprus\\OneDrive\\Desktop\\Null Class\\User Reviews.csv'
OUTPUT_DIR = 'C:\\Users\\hprus\\OneDrive\\Desktop\\Null Class\\cleaned_for_powerbi'
os.makedirs(OUTPUT_DIR, exist_ok=True)

IST = pytz.timezone('Asia/Kolkata')

def now_in_ist():
    return datetime.now(IST)

def is_now_between(start_h, start_m, end_h, end_m):
    """Inclusive start, exclusive end. Works across midnight."""
    now = now_in_ist().time()
    start = time(start_h, start_m)
    end = time(end_h, end_m)
    if start < end:
        return start <= now < end
    return now >= start or now < end

print("Environment ready. VADER available?", use_vader)
print("Files will be written to:", OUTPUT_DIR)


Environment ready. VADER available? True
Files will be written to: C:\Users\hprus\OneDrive\Desktop\Null Class\cleaned_for_powerbi


In [20]:
# Cell 2: Load datasets and quick inspection
apps = pd.read_csv(PLAYSTORE_PATH)
reviews = pd.read_csv(REVIEWS_PATH)

print("Playstore columns:", apps.columns.tolist())
print("Reviews columns  :", reviews.columns.tolist())

print("\nPlaystore sample:")
display(apps.head())
print("\nReviews sample:")
display(reviews.head())


Playstore columns: ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
Reviews columns  : ['App', 'Translated_Review', 'Sentiment', 'Sentiment_Polarity', 'Sentiment_Subjectivity']

Playstore sample:


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up



Reviews sample:


,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [21]:
# Cell 3: helper functions + clean Play Store dataset

def safe_str(x):
    return '' if pd.isna(x) else str(x).strip()

def convert_installs(s):
    s = safe_str(s)
    if s == '' or s.lower() == 'nan':
        return 0
    s = re.sub(r'[+,]', '', s)
    s = re.sub(r'[^\d]', '', s)
    try:
        return int(s)
    except:
        return 0

def convert_price(s):
    s = safe_str(s)
    if s.lower() in ['free','nan','0','']:
        return 0.0
    s = re.sub(r'[$,]', '', s)
    try:
        return float(s)
    except:
        return 0.0

def convert_size_mb(s):
    s = safe_str(s)
    if s == '' or s.lower() in ['varies with device','nan']:
        return np.nan
    s = s.replace(' ', '')
    # patterns like '19M', '1024k', '14'
    m = re.match(r'^([\d\.]+)([MmKk]|KB|kb)?$', s)
    if m:
        val = float(m.group(1))
        unit = m.group(2)
        if unit is None:
            return val
        unit = unit.lower()
        if unit in ['m']:
            return val
        if unit in ['k','kb']:
            return val / 1024.0
    try:
        return float(s)
    except:
        return np.nan

def parse_android_version(s):
    s = safe_str(s)
    m = re.search(r'(\d+(\.\d+)?)', s)
    return float(m.group(1)) if m else np.nan

# Clean Playstore
apps = apps.copy()
apps['Installs'] = apps.get('Installs', '').apply(convert_installs).astype(int)
apps['Reviews'] = pd.to_numeric(apps.get('Reviews', 0), errors='coerce').fillna(0).astype(int)
apps['Price'] = apps.get('Price', '').apply(convert_price).astype(float)
apps['Size_MB'] = apps.get('Size', '').apply(convert_size_mb)
apps['Rating'] = pd.to_numeric(apps.get('Rating', np.nan), errors='coerce')
apps = apps[apps['Rating'].notna()].copy()   # drop apps with no rating
apps['Last Updated'] = pd.to_datetime(apps.get('Last Updated', pd.NaT), errors='coerce')
apps['Update_Year'] = apps['Last Updated'].dt.year
apps['Update_Month'] = apps['Last Updated'].dt.month

# Android column detection
android_col = None
for col in ['Android Ver','Android Version','Current Ver']:
    if col in apps.columns:
        android_col = col
        break
if android_col:
    apps['Android_Version'] = apps[android_col].apply(parse_android_version)
else:
    apps['Android_Version'] = np.nan

apps['App_clean'] = apps['App'].astype(str).str.strip()
apps['App_len'] = apps['App_clean'].str.len()
apps['App_lower'] = apps['App_clean'].str.lower()
apps['App_no_S'] = ~apps['App_clean'].str.contains('S', case=False, na=False)
apps['Revenue'] = apps['Price'] * apps['Installs']

apps.drop_duplicates(inplace=True)
print("Apps cleaned:", apps.shape)


Apps cleaned: (8893, 22)


In [22]:
# Cell 4: Reviews sentiment + merge + translations + viz windows metadata

# Choose review text column
if 'Translated_Review' in reviews.columns:
    reviews['review_text'] = reviews['Translated_Review'].astype(str).fillna('')
elif 'Review' in reviews.columns:
    reviews['review_text'] = reviews['Review'].astype(str).fillna('')
else:
    reviews['review_text'] = ''

reviews = reviews[reviews['review_text'].str.strip() != ''].copy()

# Sentiment (VADER if available, else TextBlob)
if use_vader and sia is not None:
    reviews['sentiment_compound'] = reviews['review_text'].apply(lambda x: sia.polarity_scores(str(x))['compound'])
else:
    reviews['sentiment_compound'] = reviews['review_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# Subjectivity (TextBlob)
reviews['sentiment_subjectivity'] = reviews['review_text'].apply(lambda x: TextBlob(str(x)).sentiment.subjectivity)

print("Reviews cleaned:", reviews.shape)

# Merge: try exact then case-insensitive
reviews['App_rev_clean'] = reviews['App'].astype(str).str.strip() if 'App' in reviews.columns else ''
merged = pd.merge(apps, reviews, left_on='App_clean', right_on='App_rev_clean', how='left', suffixes=('_app','_rev'))

# If no merge rows, try joining on lower
if merged.shape[0] == 0 and 'App' in reviews.columns:
    reviews['App_lower'] = reviews['App'].astype(str).str.lower().str.strip()
    merged = pd.merge(apps, reviews, left_on='App_lower', right_on='App_lower', how='left', suffixes=('_app','_rev'))

# Fill missing merge columns sensibly
# Keep primary app columns where duplicates exist
for col in ['Installs','Size_MB','Rating','Reviews','Revenue','Update_Month','Update_Year','Last Updated']:
    if col not in merged.columns and col in apps.columns:
        merged[col] = apps[col]

print("Merged shape:", merged.shape)

# Translations mapping (for labels)
translations = {
    'Beauty': 'ब्यूटी',      # Hindi
    'Business': 'வணிகம்',   # Tamil
    'Dating': 'Dating (DE)', # placeholder for German label
    'Travel & Local': 'Voyage & Local',  # French placeholder
    'Productivity': 'Productividad',
    'Photography': '写真'
}
merged['Category_translated'] = merged['Category'].map(translations).fillna(merged['Category'])

# Visualization windows metadata (save to JSON)
viz_windows = {
    "task1": {"start":"15:00","end":"17:00","notes":"Grouped bar (3PM-5PM IST)"},
    "task2": {"start":"18:00","end":"20:00","notes":"Choropleth (6PM-8PM IST)"},
    "task3": {"start":"13:00","end":"14:00","notes":"Dual axis (1PM-2PM IST)"},
    "task4": {"start":"18:00","end":"21:00","notes":"Time series (6PM-9PM IST)"},
    "task5": {"start":"17:00","end":"19:00","notes":"Bubble (5PM-7PM IST)"},
    "task6": {"start":"16:00","end":"18:00","notes":"Stacked area (4PM-6PM IST)"}
}
with open(os.path.join(OUTPUT_DIR,'visualization_windows.json'),'w', encoding='utf-8') as f:
    json.dump(viz_windows, f, indent=2)

# quick check
merged.head()


Reviews cleaned: (64295, 8)
Merged shape: (107732, 31)


,App_app,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,App_rev,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,review_text,sentiment_compound,sentiment_subjectivity,App_rev_clean,Category_translated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0.0,Everyone,Art & Design,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ART_AND_DESIGN
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,Coloring book moana,A kid's excessive ads. The types ads allowed a...,Negative,-0.250,1.000000,A kid's excessive ads. The types ads allowed a...,-0.2500,1.000000,Coloring book moana,ART_AND_DESIGN
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,Coloring book moana,It bad >:(,Negative,-0.725,0.833333,It bad >:(,-0.8020,0.833333,Coloring book moana,ART_AND_DESIGN
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,Coloring book moana,like,Neutral,0.000,0.000000,like,0.3612,0.000000,Coloring book moana,ART_AND_DESIGN
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,...,Coloring book moana,NaN,NaN,NaN,NaN,nan,0.0000,0.000000,Coloring book moana,ART_AND_DESIGN


In [23]:
# Cell 5: Monthly aggregates & pivot (needed before exports)

m = merged.copy()
m['Category'] = m['Category'].astype(str).str.strip()
m['Last_Updated_Date'] = pd.to_datetime(m['Last Updated'], errors='coerce')
m['Month_Year'] = m['Last_Updated_Date'].dt.to_period('M').astype(str).fillna('Unknown')

monthly_installs = m.groupby(['Month_Year','Category']).agg(
    total_installs=('Installs','sum'),
    avg_rating=('Rating','mean'),
    total_reviews=('Reviews','sum')
).reset_index().sort_values('Month_Year')

monthly_installs['Month_Year_dt'] = pd.to_datetime(monthly_installs['Month_Year'] + '-01', errors='coerce')
monthly_installs = monthly_installs.sort_values('Month_Year_dt')

pivot_installs = monthly_installs.pivot(index='Month_Year_dt', columns='Category', values='total_installs').fillna(0)
pivot_installs_cumulative = pivot_installs.cumsum()

print("Monthly aggregates and pivot prepared.")


Monthly aggregates and pivot prepared.


In [24]:
# Cell 6: Define eligibility flags for tasks (careful: use lowercase 'sentiment_subjectivity')

# Task1 flags
m['Update_Month'] = m['Last_Updated_Date'].dt.month
m['task1_last_update_jan'] = m['Update_Month'] == 1
m['task1_size_ge_10MB'] = m['Size_MB'].fillna(0) >= 10
m['task1_avg_rating_ge_4'] = m['Rating'] >= 4.0
m['task1_eligible'] = m['task1_last_update_jan'] & m['task1_size_ge_10MB'] & m['task1_avg_rating_ge_4']

# Task2 flags (basic existence + will filter out startswith A,C,G,S in plot)
m['task2_eligible'] = (m['Category'].notna()) & (m['Category'] != '') & (m['Installs'].notna()) & (m['Installs'] > 0)

# Task3 flags
m['task3_installs_ge_10k'] = m['Installs'] >= 10_000
m['task3_revenue_ge_10k'] = m['Revenue'] >= 10_000
m['task3_android_gt_4'] = m['Android_Version'] > 4.0
m['task3_size_gt_15'] = m['Size_MB'].fillna(0) > 15
# find content rating column if exists
content_rating_col = None
for c in ['Content Rating','Content_Rating','Content rating']:
    if c in m.columns:
        content_rating_col = c
        break
if content_rating_col:
    m['task3_content_everyone'] = m[content_rating_col].astype(str).str.strip().str.lower() == 'everyone'
else:
    m['task3_content_everyone'] = False
m['task3_app_len_le_30'] = m['App_len'] <= 30
m['task3_eligible'] = (
    m['task3_installs_ge_10k'] &
    m['task3_revenue_ge_10k'] &
    m['task3_android_gt_4'] &
    m['task3_size_gt_15'] &
    m['task3_content_everyone'] &
    m['task3_app_len_le_30']
)

# Task4 flags
m['task4_reviews_gt_500'] = m['Reviews'] > 500
m['task4_app_not_start_xyz'] = ~m['App_lower'].str.match(r'^[xyz]', na=False)
m['task4_app_no_S'] = ~m['App_clean'].str.contains('S', case=False, na=False)
# Category should start with E or C or B
m['task4_cat_starts_with_ECB'] = m['Category'].astype(str).str.match(r'^[ECB]', na=False)
m['task4_eligible'] = m['task4_reviews_gt_500'] & m['task4_app_not_start_xyz'] & m['task4_app_no_S'] & m['task4_cat_starts_with_ECB']

# Task5 flags
categories_list = [
    'GAME','BEAUTY','BUSINESS','COMICS','COMMUNICATION','DATING','ENTERTAINMENT','SOCIAL','EVENTS'
]
# Use lowercase subjectivity column created earlier: 'sentiment_subjectivity'
if 'sentiment_subjectivity' not in m.columns:
    # If not present in merged, try to bring from reviews -> earlier we merged; so we just ensure column exists
    m['sentiment_subjectivity'] = m.get('sentiment_subjectivity', 0)

m['task5_rating_gt_3_5'] = m['Rating'].fillna(0) >= 3.5
m['task5_reviews_gt_500'] = m['Reviews'].fillna(0) > 500
m['task5_app_no_S'] = ~m['App_clean'].str.contains('S', case=False, na=False)
m['task5_subjectivity_gt_0_5'] = m['sentiment_subjectivity'].fillna(0) > 0.5
m['task5_installs_gt_50k'] = m['Installs'].fillna(0) > 50_000
m['task5_cat_in_list'] = m['Category'].astype(str).str.upper().isin(categories_list)
m['task5_eligible'] = (
    m['task5_rating_gt_3_5'] &
    m['task5_reviews_gt_500'] &
    m['task5_app_no_S'] &
    m['task5_subjectivity_gt_0_5'] &
    m['task5_installs_gt_50k'] &
    m['task5_cat_in_list']
)

# Task6 flags
def no_digits(x):
    return not bool(re.search(r'\d', safe_str(x)))

m['task6_avg_rating_ge_4_2'] = m['Rating'].fillna(0) >= 4.2
m['task6_app_no_digits'] = m['App_clean'].apply(no_digits)
m['task6_cat_start_T_or_P'] = m['Category'].astype(str).str.match(r'^[TP]', na=False)
m['task6_reviews_gt_1000'] = m['Reviews'].fillna(0) > 1000
m['task6_size_between_20_80'] = (m['Size_MB'].fillna(0) >= 20) & (m['Size_MB'].fillna(0) <= 80)
m['task6_eligible'] = (
    m['task6_avg_rating_ge_4_2'] &
    m['task6_app_no_digits'] &
    m['task6_cat_start_T_or_P'] &
    m['task6_reviews_gt_1000'] &
    m['task6_size_between_20_80']
)

# Quick counts
for i in range(1,7):
    col = f"task{i}_eligible"
    print(f"Task{i} eligible count:", int(m[col].sum()) if col in m.columns else 0)


Task1 eligible count: 464
Task2 eligible count: 107731
Task3 eligible count: 181
Task4 eligible count: 3714
Task5 eligible count: 4410
Task6 eligible count: 2017


In [25]:
# Cell 7: Export cleaned files for Power BI / further plotting
master_path = os.path.join(OUTPUT_DIR, 'merged_playstore_reviews_cleaned.csv')
m.to_csv(master_path, index=False, encoding='utf-8')

exports = {
    'task1_data.csv': m[m['task1_eligible']].copy(),
    'task2_data.csv': m[m['task2_eligible']].copy(),
    'task3_data.csv': m[m['task3_eligible']].copy(),
    'task4_data.csv': m[m['task4_eligible']].copy(),
    'task5_data.csv': m[m['task5_eligible']].copy(),
    'task6_data.csv': m[m['task6_eligible']].copy(),
    'monthly_installs_data.csv': monthly_installs.copy(),
    'pivot_installs_cumulative_data.csv': pivot_installs_cumulative.reset_index().rename(columns={'Month_Year_dt':'Month_Start'}).fillna(0)
}

for fname, df in exports.items():
    path = os.path.join(OUTPUT_DIR, fname)
    df.to_csv(path, index=False, encoding='utf-8')

print("Exports written to:", OUTPUT_DIR)


Exports written to: C:\Users\hprus\OneDrive\Desktop\Null Class\cleaned_for_powerbi


In [26]:
# Cell 8: plotting helpers (save/show)
def show_or_save(fig, name):
    out_html = os.path.join(OUTPUT_DIR, f'{name}.html')
    fig.write_html(out_html, full_html=True, include_plotlyjs='cdn')
    try:
        fig.show()
    except Exception:
        print(f"Saved {out_html}")


In [30]:
# Cell 9: Plotting functions (task1..task6) with correct windows & filters

# Task 1: Grouped bar (3PM-5PM)
def plot_task1(df):
    if not is_now_between(15,0,17,0):
        print("Task1: outside 15:00-17:00 IST (hidden).")
        return
    tmp = df[df['task1_eligible']].copy()
    if tmp.empty:
        print("Task1: no eligible rows.")
        return
    agg = tmp.groupby('Category').agg(
        avg_rating=('Rating','mean'),
        total_reviews=('Reviews','sum'),
        total_installs=('Installs','sum')
    ).reset_index()
    top10 = agg.sort_values('total_installs', ascending=False).head(10)
    # build combo: bars (reviews) + line (avg rating)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=top10['Category'], y=top10['total_reviews'], name='Total Reviews',
                         marker=dict(color='plum', line=dict(color='black', width=1)),
                         text=top10['total_reviews'], textposition='auto'))
    fig.add_trace(go.Scatter(x=top10['Category'], y=top10['avg_rating'], mode='lines+markers', name='Avg Rating',
                             yaxis='y2', line=dict(color='green', width=2),
                             marker=dict(size=8, color='green', line=dict(color='black', width=1))))
    fig.update_layout(title='<b>Avg Rating & Total Reviews (Top 10 by installs)</b>', yaxis=dict(title='Total Reviews'),
                      yaxis2=dict(title='Avg Rating (0-5)', overlaying='y', side='right', range=[0,5]),
                      barmode='group', template='simple_white', height=600, width=1000)
    show_or_save(fig, 'task1_grouped_bar')
    return fig

# Task 2: Choropleth (6PM-8PM) - exclude categories starting with A,C,G,S
# Task 2: Choropleth (with custom color legend box beside the map)
def plot_task2(df):
    if not is_now_between(18,0,20,0):   # Extended for your debugging
        print("Task2: outside 18:00-20:00 IST (hidden).")
        return

    tmp = df[df['task2_eligible']].copy()
    if tmp.empty or 'Category' not in tmp.columns:
        print("Task2: no eligible data.")
        return

    # Exclude categories starting with A,C,G,S
    tmp = tmp[~tmp['Category'].astype(str).str.upper().str.startswith(tuple(['A','C','G','S']))]
    if tmp.empty:
        print("Task2: no rows after excluding A,C,G,S.")
        return

    # Dummy mapping (you can extend this with real ones)
    country_map = {
        'PRODUCTIVITY': 'IN', 'TOOLS': 'US', 'FAMILY': 'GB',
        'PHOTOGRAPHY': 'CA', 'NEWS_AND_MAGAZINES': 'AU',
        'GAME': 'IN', 'BEAUTY': 'IN', 'BUSINESS': 'US', 'DATING': 'GB'
    }
    tmp['Country'] = tmp['Category'].str.upper().map(country_map)

    # Safe ISO2 → ISO3
    import pycountry
    def safe_iso2_to_iso3(alpha2):
        try:
            return pycountry.countries.get(alpha_2=alpha2).alpha_3
        except Exception:
            return None
    tmp['ISO3'] = tmp['Country'].apply(lambda x: safe_iso2_to_iso3(x) if pd.notna(x) else None)

    agg_df = tmp.groupby(['ISO3','Category'], as_index=False)['Installs'].sum()
    if agg_df.empty:
        print("Task2: aggregation empty.")
        return

    top5_categories = agg_df.groupby('Category')['Installs'].sum().nlargest(5).index.tolist()
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    category_colors = dict(zip(top5_categories, colors))

    # --- Choropleth traces ---
    fig = go.Figure()
    for cat in top5_categories:
        cat_df = agg_df[agg_df['Category'] == cat].copy()
        if cat_df.empty:
            continue
        z_values = [1 if v > 1_000_000 else 0 for v in cat_df['Installs']]
        hover_text = [f"{row['ISO3']}<br>{cat} Installs: {int(row['Installs']):,}" for _, row in cat_df.iterrows()]
        fig.add_trace(go.Choropleth(
            locations=cat_df['ISO3'],
            z=z_values,
            colorscale=[[0, 'lightgray'], [1, category_colors[cat]]],
            zmin=0, zmax=1,
            text=hover_text,
            hoverinfo='text',
            showscale=False,  # disable default colorbar
            name=cat
        ))

    # --- Custom legend box (using Scatter markers) ---
    legend_y = 0.95
    for cat, color in category_colors.items():
        fig.add_trace(go.Scatter(
            x=[None], y=[None],
            mode='markers',
            marker=dict(size=15, color=color, symbol='square'),
            legendgroup=cat,
            showlegend=True,
            name=f"{cat} (>1M Installs)"
        ))

    # --- Dropdowns for category selection ---
    buttons = []
    for i, cat in enumerate(top5_categories):
        visible = [False]*len(top5_categories) + [True]*len(top5_categories)  # keep legend visible
        visible[i] = True
        buttons.append(dict(
            label=cat,
            method='update',
            args=[{'visible': visible},
                  {'title': f"Global Installs for {cat} (>1M)"}]
        ))
    buttons.insert(0, dict(
        label='All',
        method='update',
        args=[{'visible':[True]*len(top5_categories) + [True]*len(top5_categories)},
              {'title':'Global Installs (Top 5 Categories)'}]
    ))

    fig.update_layout(
        title=dict(
            text='<b>Choropleth of App Installs by Country (Top 5 Categories)</b>',
            x=0.5, y=0.95, xanchor='center', font=dict(size=18)
        ),
        updatemenus=[dict(active=0, buttons=buttons, x=1.15, y=1.05)],
        geo=dict(showframe=False, showcoastlines=True, projection_type='natural earth'),
        legend=dict(
            title='<b>Category Color Legend</b>',
            x=1.02, y=0.5,
            bgcolor='rgba(255,255,255,0.6)',
            bordercolor='black',
            borderwidth=1
        ),
        margin=dict(r=200),  # extra space for legend box
        template='simple_white',
        height=600,
        width=1100
    )

    show_or_save(fig, 'task2_choropleth_with_legend')
    return fig


# Task 3: Dual-axis (1PM-2PM) - free vs paid avg installs and revenue
def plot_task3(df):
    if not is_now_between(13,0,14,0):
        print("Task3: outside 13:00-14:00 IST (hidden).")
        return
    tmp = df[df['task3_eligible']].copy()
    if tmp.empty:
        print("Task3: no eligible data.")
        return
    # Ensure Type column exists (Free/Paid). If not, create from Price
    if 'Type' not in tmp.columns:
        tmp['Type'] = tmp['Price'].apply(lambda p: 'Free' if p==0 else 'Paid')
    top3 = tmp.groupby('Category')['Installs'].sum().nlargest(3).index.tolist()
    tmp = tmp[tmp['Category'].isin(top3)]
    if tmp.empty:
        print("Task3: no rows in top3 categories after filtering.")
        return
    agg = tmp.groupby(['Category','Type']).agg(avg_installs=('Installs','mean'), avg_revenue=('Revenue','mean')).reset_index()
    fig = go.Figure()
    # bars (avg installs)
    for t in agg['Type'].unique():
        sub = agg[agg['Type']==t]
        color = 'gold' if t=='Free' else 'thistle'
        fig.add_trace(go.Bar(x=sub['Category'], y=sub['avg_installs'], name=f'Avg Installs ({t})', marker=dict(color=color, line=dict(color='black', width=1))))
    # lines (avg revenue) on y2
    for t in agg['Type'].unique():
        sub = agg[agg['Type']==t]
        line_color = 'mediumseagreen' if t=='Free' else 'orchid'
        fig.add_trace(go.Scatter(x=sub['Category'], y=sub['avg_revenue'], mode='lines+markers', name=f'Avg Revenue ({t})', yaxis='y2', line=dict(color=line_color, width=2), marker=dict(size=8, color=line_color, line=dict(color='black', width=1))))
    fig.update_layout(title='<b>Avg Installs vs Avg Revenue (Free vs Paid) - Top 3 Categories</b>', yaxis=dict(title='Avg Installs'), yaxis2=dict(title='Avg Revenue', overlaying='y', side='right'), barmode='group', template='simple_white', height=600, width=1000)
    show_or_save(fig, 'task3_dual_axis')
    return fig

# Task 4: Time series (6PM-9PM) with shading when MoM > 20%
def plot_task4(df):
    if not is_now_between(18,0,21,0):
        print("Task4: outside 18:00-21:00 IST (hidden).")
        return
    tmp = df[df['task4_eligible']].copy()
    if tmp.empty:
        print("Task4: no eligible rows.")
        return
    tmp['Month_Year'] = tmp['Last_Updated_Date'].dt.to_period('M').astype(str)
    ts = tmp.groupby(['Month_Year','Category']).agg(total_installs=('Installs','sum')).reset_index()
    ts['Month_dt'] = pd.to_datetime(ts['Month_Year'] + '-01', errors='coerce')
    fig = go.Figure()
    pastel_colors = ['lightgreen','plum','lightblue','salmon','khaki','mediumorchid','gold']
    for i, c in enumerate(ts['Category'].unique()):
        sub = ts[ts['Category']==c].sort_values('Month_dt')
        color = pastel_colors[i % len(pastel_colors)]
        fig.add_trace(go.Scatter(x=sub['Month_dt'], y=sub['total_installs'], mode='lines+markers', name=c, line=dict(color=color, width=2), marker=dict(size=6, color=color, line=dict(color='black', width=1))))
        sub = sub.copy()
        sub['mom'] = sub['total_installs'].pct_change()
        for _, row in sub.iterrows():
            if pd.notna(row['mom']) and row['mom'] > 0.2:
                start = row['Month_dt']
                end = start + pd.DateOffset(months=1)
                fig.add_vrect(x0=start, x1=end, fillcolor="LightSalmon", opacity=0.2, layer="below", line_width=0)
    fig.update_layout(title='<b>Monthly installs by category (Shaded: MoM > 20%)</b>', xaxis_title='Month', yaxis_title='Installs', template='simple_white', height=600, width=1000)
    show_or_save(fig, 'task4_time_series')
    return fig

# Task 5: Bubble chart (5PM-7PM) - filter by eligibility & subjectivity
# Task 5: Bubble chart (5PM–7PM) - filter by eligibility & subjectivity
def plot_task5(df):
    if not is_now_between(17, 0, 19, 0):   # extended till 11 PM for testing
        print("Task5: outside 17:00-19:00 IST (hidden).")
        return

    tmp = df[df['task5_eligible']].copy()
    if tmp.empty:
        print("Task5: no eligible rows.")
        return

    # Ensure category and clean columns
    tmp['Category_upper'] = tmp['Category'].astype(str).str.upper()

    # ✅ Build clear, accurate hover text for every point
    tmp['hover_text'] = (
        "<b>App:</b> " + tmp['App_clean'].astype(str) +
        "<br><b>Category:</b> " + tmp['Category'].astype(str) +
        "<br><b>Installs:</b> " + tmp['Installs'].astype(int).astype(str) +
        "<br><b>Size (MB):</b> " + tmp['Size_MB'].astype(str) +
        "<br><b>Rating:</b> " + tmp['Rating'].astype(str)
    )

    # Keep same logic — bubble size based on installs
    fig = px.scatter(
        tmp,
        x='Size_MB',
        y='Rating',
        size='Installs',                    # true install-based size
        color='Category',
        hover_name='App_clean',
        hover_data={'Installs': True, 'Size_MB': True, 'Rating': True, 'Category': True},
        title='<b>Bubble Chart: App Size vs Rating (Bubble = Installs)</b>',
        size_max=90,                        # enlarged for visibility
        opacity=0.8,
        template='simple_white'
    )

    # Replace hover info with custom text
    for trace in fig.data:
        trace.hovertemplate = "%{text}<extra></extra>"
    fig.update_traces(text=tmp['hover_text'])

    # Highlight Game category in pink
    for trace in fig.data:
        if trace.name.upper() == 'GAME':
            trace.marker.color = 'pink'

    # Layout styling
    fig.update_layout(
        xaxis_title='App Size (MB)',
        yaxis_title='Average Rating',
        legend=dict(
            title='<b>App Categories</b>',
            bgcolor='rgba(255,255,255,0.6)',
            bordercolor='black',
            borderwidth=1
        ),
        height=650,
        width=1000
    )

    show_or_save(fig, 'task5_bubble_correct_tooltip')
    return fig

    

# Task 6: Stacked area cumulative installs (4PM-6PM) with intensity highlight for MoM>25%
def plot_task6(df):
    if not is_now_between(16,0,18,0):
        print("Task6: outside 16:00-18:00 IST (hidden).")
        return
    pivot = pivot_installs_cumulative.reset_index().rename(columns={'Month_Year_dt':'Month_Start'}).copy()
    pivot['Month_Start'] = pivot['Month_Year_dt'] if 'Month_Year_dt' in pivot.columns else pivot['Month_Start']
    pivot['Month_Start'] = pd.to_datetime(pivot['Month_Year_dt']) if 'Month_Year_dt' in pivot.columns else pd.to_datetime(pivot['Month_Start'])
    pivot = pivot.set_index('Month_Year_dt') if 'Month_Year_dt' in pivot.columns else pivot.set_index('Month_Start')
    # compute MoM pct change across each category
    pivot_diff = pivot.pct_change()
    fig = go.Figure()
    for col in pivot.columns:
        y = pivot[col].values
        fig.add_trace(go.Scatter(x=pivot.index, y=y, stackgroup='one', name=col))
        # highlight months where pct change > 0.25 for this category
        highlight_idx = pivot_diff[pivot_diff[col] > 0.25].index
        for start in highlight_idx:
            end = start + pd.DateOffset(months=1)
            fig.add_vrect(x0=start, x1=end, fillcolor="rgba(255,0,0,0.12)", opacity=0.2, layer="below", line_width=0)
    fig.update_layout(title='<b>Cumulative installs (stacked area) - highlight MoM>25%</b>', xaxis_title='Month', yaxis_title='Cumulative Installs', template='simple_white', height=600, width=1000)
    show_or_save(fig, 'task6_stacked_area')
    return fig


In [31]:
# Cell 10: Run plotting functions (these will respect IST windows)
plot_task1(m)
plot_task2(m)
plot_task3(m)
plot_task4(m)
plot_task5(m)
plot_task6(m)
print("Plot generation attempted (HTML snapshots saved in OUTPUT_DIR where applicable).")


Task1: outside 15:00-17:00 IST (hidden).
Task2: outside 18:00-20:00 IST (hidden).
Task3: outside 13:00-14:00 IST (hidden).
Task4: outside 18:00-21:00 IST (hidden).
Task5: outside 17:00-19:00 IST (hidden).
Task6: outside 16:00-18:00 IST (hidden).
Plot generation attempted (HTML snapshots saved in OUTPUT_DIR where applicable).


In [29]:
# ================================================================
# DASHBOARD GENERATION SECTION  ✅ Dash 3+ Compatible
# ================================================================
import datetime
import pytz
import plotly.io as pio
from dash import Dash, html, dcc

# ------------------------------------------------
# Helper function to check IST time window
# ------------------------------------------------
def is_now_between(start_hour, start_min, end_hour, end_min):
    """Check if current time is between given hours (IST)."""
    tz_ist = pytz.timezone("Asia/Kolkata")
    now = datetime.datetime.now(tz_ist)
    start = now.replace(hour=start_hour, minute=start_min, second=0, microsecond=0)
    end = now.replace(hour=end_hour, minute=end_min, second=0, microsecond=0)
    if end < start:  # handle midnight crossover
        end += datetime.timedelta(days=1)
    return start <= now <= end


# ------------------------------------------------
# Create Dash app
# ------------------------------------------------
app = Dash(__name__)
app.title = "Google Play Store Data Analytics Dashboard"

figures = []

# ------------------------------------------------
# Conditional chart generation based on time window
# ------------------------------------------------
try:
    if is_now_between(15, 0, 17, 0):  # Task 1 (3PM–5PM, extended)
        fig1 = plot_task1(m)
        figures.append(html.Div([
            html.H3("Task 1: Avg Rating vs Total Reviews (Top 10 Categories)",
                    style={'textAlign': 'center'}),
            dcc.Graph(figure=fig1)
        ]))

    if is_now_between(18, 0, 20, 0):  # Task 2 (6PM–8PM, extended)
        fig2 = plot_task2(m)
        figures.append(html.Div([
            html.H3("Task 2: Global Installs Choropleth (Top 5 Categories)",
                    style={'textAlign': 'center'}),
            dcc.Graph(figure=fig2)
        ]))

    if is_now_between(13, 0, 14, 0):  # Task 3 (1PM–2PM, extended)
        fig3 = plot_task3(m)
        figures.append(html.Div([
            html.H3("Task 3: Dual-Axis Free vs Paid Installs & Revenue",
                    style={'textAlign': 'center'}),
            dcc.Graph(figure=fig3)
        ]))

    if is_now_between(18, 0, 21, 0):  # Task 4 (6PM–9PM, extended)
        fig4 = plot_task4(m)
        figures.append(html.Div([
            html.H3("Task 4: Time Series — Monthly Install Growth",
                    style={'textAlign': 'center'}),
            dcc.Graph(figure=fig4)
        ]))

    if is_now_between(17, 0, 19, 0):  # Task 5 (5PM–7PM, extended)
        fig5 = plot_task5(m)
        figures.append(html.Div([
            html.H3("Task 5: Bubble — Size vs Rating (Bubble = Installs)",
                    style={'textAlign': 'center'}),
            dcc.Graph(figure=fig5)
        ]))

    if is_now_between(16, 0, 18, 0):  # Task 6 (4PM–6PM, extended)
        fig6 = plot_task6(m)
        figures.append(html.Div([
            html.H3("Task 6: Stacked Area — Cumulative Installs Over Time",
                    style={'textAlign': 'center'}),
            dcc.Graph(figure=fig6)
        ]))

except Exception as e:
    print("⚠️ Error while generating figures:", e)


# ------------------------------------------------
# Default message if no charts active
# ------------------------------------------------
if not figures:
    figures = [html.Div([
        html.H2("No visualizations available at this time window.",
                style={'textAlign': 'center', 'marginTop': '100px', 'color': '#555'})
    ])]


# ------------------------------------------------
# Dashboard layout
# ------------------------------------------------
app.layout = html.Div([
    html.H1("Google Play Store Data Analytics Dashboard",
            style={'textAlign': 'center', 'color': '#1a237e', 'marginBottom': '30px'}),
    html.Div(figures)
])


# ------------------------------------------------
# Safe dashboard export (works without Dash server)
# ------------------------------------------------
dashboard_path = r"C:\Users\hprus\OneDrive\Desktop\Null Class\cleaned_for_powerbi\dashboard.html"

try:
    html_parts = [
        "<html><head><title>Google Play Store Data Analytics Dashboard</title></head><body>",
        "<h1 style='text-align:center;color:#1a237e;'>Google Play Store Data Analytics Dashboard</h1>"
    ]

    # Export each figure individually to HTML fragment
    for i, fig_container in enumerate(figures, start=1):
        try:
            for component in fig_container.children:
                if isinstance(component, dcc.Graph):
                    fig = component.figure
                    if fig is not None:
                        html_parts.append(f"<h3 style='text-align:center;'>Visualization {i}</h3>")
                        html_parts.append(pio.to_html(fig, include_plotlyjs='cdn', full_html=False))
                    else:
                        print(f"⚠️ Figure {i} returned None (check your plot_task{i} function).")
        except Exception as inner_err:
            print(f"⚠️ Could not export figure {i}: {inner_err}")

    html_parts.append("</body></html>")

    # Write full HTML dashboard file
    with open(dashboard_path, "w", encoding="utf-8") as f:
        f.write("\n".join(html_parts))

    print(f"\n✅ Dashboard successfully exported to:\n{dashboard_path}")

except Exception as e:
    print(f"❌ Dashboard export failed: {e}")





✅ Dashboard successfully exported to:
C:\Users\hprus\OneDrive\Desktop\Null Class\cleaned_for_powerbi\dashboard.html
